In [ ]:
import numpy as np
from scipy.stats import norm
from scipy import stats as ss

In [23]:
def covariance_matrix(sigma, rho):
  """
  Compute the covariance matrix given volatilities (sigma) and correlation matrix (rho).

  Parameters:
  - sigma: array of volatilities of each stock
  - rho: correlation matrix of Brownian motions

  Output:
  - SIGMA: covariance matrix
  """
  SIGMA = np.dot(np.diag(sigma), np.dot(rho, np.diag(sigma)))
  return SIGMA

def MC_call_on_min_VG_pricer(S0, K, r, T, sigma, rho, theta, nu, B, N, M, alpha_conf, seed):
  """
  Compute the price of a call on min option under variance gamma for a system of d stocks.

  Parameters:
  - S0: vector of initial stock prices
  - K: strike price
  - r: risk-free interest rate
  - T: maturity time
  - sigma: vector of volatilities
  - rho: correlation matrix of Brownian motions
  - B: number of batches
  - N: number of time steps of forward Euler discretization
  - M: number of MC sample paths
  - alpha_conf: confidence level
  - seed: random seed

  Returns:
  - price_estimate: estimated price of the option using Monte Carlo
  - MC_stat_error: statistical error of the Monte Carlo estimation
  """
  np.random.seed(seed)
  dimension = len(S0)  # number of stocks
  SIGMA = covariance_matrix(sigma, rho)  # Construct the covariance matrix
  L = np.linalg.cholesky(rho)  # Cholesky decomposition of the correlation matrix
  price_estimates_per_batch = np.zeros(B)  # MC price estimates per batch
  price_stds_per_batch = np.zeros(B)  # MC standard deviation per batch
  N = 1  # Exact simulation scheme
  dt = T / N  # size of time sub-interval between two consecutive increments
  alpha = dt/nu #shape parameter of the Gamma distribution
  beta = 1/nu #rate parameter = inverse scale parameter of the Gamma distribution
  mu =(1/nu)*np.log( 1 - np.multiply(nu,theta) - np.multiply(0.5*nu,np.diag(SIGMA)) ) # martingale correction term for d stocks.

  for b in range(B):
    X = np.zeros((M, N + 1, dimension )) # Contains all sample paths of each time step for each stock: M_paths x N_steps x d
    Z = np.random.multivariate_normal(mean  = np.zeros(dimension), cov = np.identity(dimension), size = (M,N)) # samples from independent standard normal distribution
    G = ss.gamma.rvs(a = alpha , scale = 1/beta , size = (M,N) ) # generation of the common Gamma clock for all stocks at each sample path and time step.

    for n in range(N):
      for m in range(M):
        W_correlated = L @ Z[m][n] # correlating Brownian motions
        X[m,n+1,:] = X[m,n,:] + (r + mu) * dt + theta * G[m][n] + sigma * np.sqrt(G[m][n]) * W_correlated #increment of the VG process

    XT = X[:, -1, :]  # asset log-price at the final time for each sample path
    ST = S0 * np.exp(XT)  # Stock prices at maturity

    payoff_evals = np.exp(-r * T) * np.maximum(np.min(ST, axis=1) - K, 0)  # Compute payoffs for each path
    price_estimates_per_batch[b] = np.mean(payoff_evals)  # MC price estimate per batch
    price_stds_per_batch[b] = np.std(payoff_evals)  # MC standard deviation per batch

  price_estimate = np.mean(price_estimates_per_batch)  # Final MC estimate
  C_alpha = norm.ppf(1 - alpha_conf / 2)
  MC_stat_error = C_alpha * np.mean(price_stds_per_batch) / np.sqrt(M * B)  # Final MC statistical error
  return price_estimate, MC_stat_error

def MC_basket_put_VG_pricer(S0, K, r, T, sigma, rho, theta, nu,  B, N, M, alpha_conf, seed):
  """
  Compute the price of a basket put option under variance gamma for a system of d stocks.

  Parameters:
  - S0: vector of initial stock prices
  - K: strike price
  - r: risk-free interest rate
  - T: maturity time
  - sigma: vector of volatilities
  - rho: correlation matrix of Brownian motions
  - B: number of batches
  - N: number of time steps
  - M: number of MC sample paths
  - alpha_conf: confidence level
  - seed: random seed

  Returns:
  - price_estimate: estimated price of the option
  - MC_stat_error: statistical error of the Monte Carlo estimation
  """
  np.random.seed(seed)
  dimension = len(S0)  # number of stocks
  SIGMA = covariance_matrix(sigma, rho)  # Construct the covariance matrix
  L = np.linalg.cholesky(rho)  # Cholesky decomposition of the correlation matrix
  price_estimates_per_batch = np.zeros(B)  # MC price estimates per batch
  price_stds_per_batch = np.zeros(B)  # MC standard deviation per batch
  N = 1  # Exact simulation scheme
  dt = T / N  # size of time sub-interval between two consecutive increments
  alpha = dt/nu #shape parameter of the Gamma distribution
  beta = 1/nu #rate parameter = inverse scale parameter of the Gamma distribution
  mu =(1/nu)*np.log( 1 - np.multiply(nu,theta) - np.multiply(0.5*nu,np.diag(SIGMA)) ) # martingale correction term for d stocks.

  for b in range(B):
    X = np.zeros((M, N + 1, dimension )) # Contains all sample paths of each time step for each stock: M_paths x N_steps x d
    Z = np.random.multivariate_normal(mean  = np.zeros(dimension), cov = np.identity(dimension), size = (M,N)) # samples from independent standard normal distribution
    G = ss.gamma.rvs(a = alpha , scale = 1/beta , size = (M,N) ) # generation of the common Gamma clock for all stocks at each sample path and time step.

    for n in range(N):
      for m in range(M):
        W_correlated = L @ Z[m][n] # correlating Brownian motions
        X[m,n+1,:] = X[m,n,:] + (r + mu) * dt + theta * G[m][n] + sigma * np.sqrt(G[m][n]) * W_correlated #increment of the VG process

    XT = X[:, -1, :]  # asset log-price at the final time for each sample path
    ST = S0 * np.exp(XT)  # Stock prices at maturity

    payoff_evals = np.exp(-r * T) * np.maximum(K - np.mean(ST, axis = 1), 0)  # Compute payoffs for each path
    price_estimates_per_batch[b] = np.mean(payoff_evals)  # MC price estimate per batch
    price_stds_per_batch[b] = np.std(payoff_evals)  # MC standard deviation per batch

  price_estimate = np.mean(price_estimates_per_batch)  # Final MC estimate
  C_alpha = norm.ppf(1 - alpha_conf / 2)
  MC_stat_error = C_alpha * np.mean(price_stds_per_batch) / np.sqrt(M * B)  # Final MC statistical error
  return price_estimate, MC_stat_error

# Pricing call on min options

In [26]:
############### Model and payoff parameters ###############
#Payoff Parameters
dimension = 4
S0= 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#VG Model Parameters
sigma = 0.4*np.ones(dimension)
theta = -0.3*np.ones(dimension)
nu = 0.2
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)
############### MC parameters ###############
B = 1 # number of batches
N = 1 # exact simulation scheme
M = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
MC_price_estimate, MC_stat_error = MC_call_on_min_VG_pricer(S0, K, r, T, sigma, rho, theta, nu, B, N, M, alpha_conf, seed)
print("MC price estimate =", round(MC_price_estimate, 5),"\nMC relative statistical error = ", round(MC_stat_error / MC_price_estimate,4) )

MC price estimate = 0.55123 
MC relative statistical error =  0.0343


# Pricing basket put options

In [27]:
############### Model and payoff parameters ###############
#Payoff Parameters
dimension = 4
S0= 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#VG Model Parameters
sigma = 0.4*np.ones(dimension)
theta = -0.3*np.ones(dimension)
nu = 0.2
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)
############### MC parameters ###############
B = 1 # number of batches
N = 1 # exact simulation scheme
M = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
MC_price_estimate, MC_stat_error = MC_basket_put_VG_pricer(S0, K, r, T, sigma, rho, theta, nu,  B, N, M, alpha_conf, seed)
print("MC price estimate =", round(MC_price_estimate, 5),"\nMC relative statistical error = ", round(MC_stat_error / MC_price_estimate,4) )

MC price estimate = 8.7898 
MC relative statistical error =  0.0089
